In [1]:
import re
import pandas as pd
from datetime import datetime, timedelta
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import json_tuple
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
import numpy as np
import json
import os
from calendar import monthrange
import pymysql
import sys 
import requests
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession .builder .master('yarn').config('spark.executor.memory','2G')\
.appName('es_mysql')\
.config('spark.driver.cores','1').config('spark.driver.memory','2G')\
.config('spark.executor.cores', '1')\
.config('spark.yarn.queue','mlbigdata')\
.enableHiveSupport()\
.getOrCreate() 

In [3]:
with open("/data/khanhnq01/file/config.json") as json_data_file:
    cfg = json.load(json_data_file)

In [4]:
def get_df(app):
    df_hdf=[]
    date_index = pd.date_range(start=date_start,end=date_end)
    for i in date_index:
        dt = i.date()
        path = cfg['hdfs_log_path']['master1'] + str(dt)
        df_dt = spark.read.parquet(f"{path}").where((F.col('appId').contains(app)))
        df_dt = df_dt.withColumn('date',F.lit(dt))
        df_hdf.append(df_dt)
    i = 0
    df_spark = df_hdf[0]
    while i < len(df_hdf)-1:
        df_spark = df_spark.unionAll(df_hdf[i+1])
        i+=1
    df_new = df_spark
    df_action = df_new.select('date','eventId','provider','userId')
    return df_action

In [5]:
date_end = '2022-06-14'
date_start = '2022-05-14'
df = get_df('viva') 
df.show()

+----------+-------+--------+-----------------+
|      date|eventId|provider|           userId|
+----------+-------+--------+-----------------+
|2022-05-14|      7|      -1|18477149983981644|
|2022-05-14|     22|      26|18140916401121745|
|2022-05-14|     25|      -1|18622656454465292|
|2022-05-14|      7|      -1|18477149983981644|
|2022-05-14|     11|      -1|69764280505319987|
|2022-05-14|   2008|      -1|18022740485541629|
|2022-05-14|     32|      -1|67867142583206745|
|2022-05-14|     22|      51|34742365042148063|
|2022-05-14|     32|      -1|48454592159985325|
|2022-05-14|   2008|      -1|63737400014370874|
|2022-05-14|      2|      -1|31626662570017485|
|2022-05-14|     22|       6|65471857748851746|
|2022-05-14|     22|      26|13613824577097438|
|2022-05-14|      7|      -1|18477149983981644|
|2022-05-14|      2|      -1|18520014728318526|
|2022-05-14|   2008|      -1|18022740485541629|
|2022-05-14|     32|      -1|75001925987206845|
|2022-05-14|      7|      -1|18477149983

In [13]:
df2=df.filter((df['provider']==5)|(df['provider']==60))

In [14]:
df_pd=df2.toPandas()
df_pd

,date,eventId,provider,userId
0,2022-05-14,22,5,34647140562069573
1,2022-05-14,22,5,105895031465579068
2,2022-05-14,22,5,36136688108339867
3,2022-05-14,22,60,18527905363894597
4,2022-05-14,14,60,35563920604168709
...,...,...,...,...
143435,2022-06-14,22,60,34831358441502901
143436,2022-06-14,22,5,37896180753187306
143437,2022-06-14,14,60,18485317533540457
143438,2022-06-14,22,5,18516632804639527


In [15]:
df_pd['provider']=df_pd['provider'].astype('int')
df_pd['date']= pd.to_datetime(df_pd['date'])

In [17]:
df_provider1=df_pd[(df_pd['date'] > '2022-05-31')]
df_provider1['userId'].nunique()

6401

#### Cho thấy là các user có provider 5,60 đều là các user cũ ,nhóm user này có 6401 user, từ 1/6-14/6

In [18]:
df_provider1['eventId'].value_counts()

22      52993
14      18628
4         502
4023      397
3001      387
2         386
4002      369
4025      239
21        166
3007      144
20        129
3003       55
4028       11
4004        4
4029        3
4003        2
4015        1
Name: eventId, dtype: int64

In [22]:
df_provider2=df_pd[(df_pd['date']>'2022-05-13')&(df_pd['date']<'2022-06-01')]

#### Active,action của nhóm user tiếp cận provider 5,60 tăng  từ 14/5-1/6 :
+ Chủ yếu là cuộn qua bài viết
+ Sau đó là đọc sapo và click xem thêm

In [21]:
final_df = df_provider2.merge(df_provider1,how='inner',on='userId')
final_df['eventId_x'].value_counts()

22      3623308
14      1312544
3001      28838
4         27150
4025      26394
2         24975
4023      13477
21        11082
20        10390
4002       8748
3007       4751
3003        801
4029        254
4003        244
4028         34
4010         30
4006          4
Name: eventId_x, dtype: int64